In [1]:
import pandas as pd

In [3]:
parametros_provas = pd.read_parquet('parametros_provas.parquet')
parametros_provas.TP_LINGUA.fillna(-1,inplace=True)

In [12]:
df1 = pd.read_csv('prova_enem_2019_etl.csv',sep=';')

df_espanhol = df1[df1['TP_LINGUA'] == 1]

df_espanhol['TX_RESPOSTAS_LC'] = df_espanhol['TX_RESPOSTAS_LC'].str.replace('9','')

df_espanhol_etl = df1[['NU_INSCRICAO','CO_PROVA_LC','TX_RESPOSTAS_LC']]    
df_espanhol_etl[[c for c in range(1,46)]] = 0

df_espanhol_etl[[c for c in range(1,46)]] = df_espanhol['TX_RESPOSTAS_LC'].apply(lambda x: pd.Series(list(x)))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [133]:
def agrupaErros(df):
    colunas_tratadas = []
    for coluna in df.columns:
        posicao_sub = coluna.find('_')
        if posicao_sub != -1:
            colunas_tratadas.append(coluna.replace(coluna[posicao_sub:],''))
        else:
            colunas_tratadas.append(coluna)

    df.columns = colunas_tratadas

    df = df.groupby(by=df.columns, axis=1).sum()
    
    return df

def agregaProvas(dic):
    df_final = pd.DataFrame()
    for cod in dic.keys():
        df_final = pd.concat([df_final,dic[cod]])
        df_final.fillna(0,inplace=True)
    return df_final

In [134]:
provas_tratadas = {}

cd_provas = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == 1)]['CO_PROVA'].unique()

co_provas = list(df_espanhol_etl.CO_PROVA_LC.unique())

co_provas = [int(prova) for prova in co_provas]
for co_prova in co_provas:
    print(co_prova)
    df_espanhol_etl_1 = pd.DataFrame()
    df_espanhol_etl_1 = df_espanhol_etl[df_espanhol_etl['CO_PROVA_LC'] == co_prova][['NU_INSCRICAO']]
    repetidas = {}
    for posicao in range(1,46):
        if co_prova in cd_provas:
            if posicao <=5:
                habilidade = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == 1) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['Descricao_Habilidade'].values[0]

                gabarito = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == 1) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['TX_GABARITO'].values[0]
            else:
                habilidade = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == -1) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['Descricao_Habilidade'].values[0]

                gabarito = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == -1) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['TX_GABARITO'].values[0]
                
            if habilidade not in list(df_espanhol_etl_1.columns):
                df_espanhol_etl_1[habilidade] = df_espanhol_etl[df_espanhol_etl['CO_PROVA_LC'] == co_prova][posicao]\
                .apply(lambda x: 1 if x == gabarito else 0)
                repetidas[habilidade] = 1
            else:
                cont  = repetidas[habilidade]+1
                df_espanhol_etl_1[f'{habilidade}_{cont}'] = df_espanhol_etl[df_espanhol_etl['CO_PROVA_LC'] == co_prova][posicao]\
                .apply(lambda x: 1 if x == gabarito else 0)
                repetidas[habilidade] +=1
    
    df_espanhol_etl_1.set_index('NU_INSCRICAO',inplace=True)
    
    df_espanhol_etl_1 = agrupaErros(df_espanhol_etl_1)
    
    provas_tratadas[co_prova] = df_espanhol_etl_1 
    
df_espanhol_final = agregaProvas(provas_tratadas)

513
512
514
511
525
521
554
553
552
551
565


In [136]:
df_ingles = df1[df1['TP_LINGUA'] == 0]

df_ingles['TX_RESPOSTAS_LC'] = df_ingles['TX_RESPOSTAS_LC'].str.replace('9','')

df_ingles_etl = df1[['NU_INSCRICAO','CO_PROVA_LC','TX_RESPOSTAS_LC']]    
df_ingles_etl[[c for c in range(1,46)]] = 0

df_ingles_etl[[c for c in range(1,46)]] = df_ingles['TX_RESPOSTAS_LC'].apply(lambda x: pd.Series(list(x)))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [137]:
provas_tratadas = {}

cd_provas = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == 0)]['CO_PROVA'].unique()

co_provas = list(df_ingles_etl.CO_PROVA_LC.unique())

co_provas = [int(prova) for prova in co_provas]
for co_prova in co_provas:
    print(co_prova)
    df_ingles_etl_1 = pd.DataFrame()
    df_ingles_etl_1 = df_ingles_etl[df_ingles_etl['CO_PROVA_LC'] == co_prova][['NU_INSCRICAO']]
    repetidas = {}
    for posicao in range(1,46):
        if co_prova in cd_provas:
            if posicao <=5:
                habilidade = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == 0) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['Descricao_Habilidade'].values[0]

                gabarito = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == 0) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['TX_GABARITO'].values[0]
            else:
                habilidade = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == -1) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['Descricao_Habilidade'].values[0]

                gabarito = parametros_provas[(parametros_provas['SG_AREA'] == 'LC') & (parametros_provas['TP_LINGUA'] == -1) \
                             & (parametros_provas['CO_PROVA'] == co_prova) & (parametros_provas['CO_POSICAO'] == posicao)]\
            ['TX_GABARITO'].values[0]
                
            if habilidade not in list(df_ingles_etl_1.columns):
                df_ingles_etl_1[habilidade] = df_ingles_etl[df_ingles_etl['CO_PROVA_LC'] == co_prova][posicao]\
                .apply(lambda x: 1 if x == gabarito else 0)
                repetidas[habilidade] = 1
            else:
                cont  = repetidas[habilidade]+1
                df_ingles_etl_1[f'{habilidade}_{cont}'] = df_ingles_etl[df_ingles_etl['CO_PROVA_LC'] == co_prova][posicao]\
                .apply(lambda x: 1 if x == gabarito else 0)
                repetidas[habilidade] +=1
    
    df_ingles_etl_1.set_index('NU_INSCRICAO',inplace=True)
    
    df_ingles_etl_1 = agrupaErros(df_ingles_etl_1)
    
    provas_tratadas[co_prova] = df_ingles_etl_1 
    
df_ingles_final = agregaProvas(provas_tratadas)

514
513
512
511
525
521
554
553
551
552
565


In [138]:
df_portugues_total = pd.concat([df_espanhol_final,df_ingles_final])

In [141]:
df_portugues_total.drop_duplicates(inplace=True)

In [142]:
df_portugues_total.to_parquet('erroshabilidadeslc_grp.parquet')